In [1]:
#basic python packages for plotting and array management
import numpy as np
import matplotlib.pyplot as plt
import random

#for data import
import pandas as pd
from __future__ import division
import os
import sys
from importlib import reload
reload(sys)

#Preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

#Classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier

#Ensemble methods and crossvalidation
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier,RandomForestClassifier, VotingClassifier

#PostProcessing
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import time

# Loading DATA

In [130]:
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

In [131]:
products.head()

,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,728257.0,Homme,Classique,Automne/Hiver,Saisonnier,66.0,DÃ©tente,Standard,17267.0,Baskets,...,NaN,0.0,30.50,0.0,Montantes,NaN,NaN,NaN,False,Prenez une taille en dessous de votre pointure...
1,806356.0,Femme,ND,Automne/Hiver,Saisonnier,842.0,ND,Standard,30824.0,Baskets,...,0.0,0.0,43.00,0.0,Montantes,NaN,NaN,NaN,True,Prenez votre pointure habituelle
2,768790.0,Femme,ND,Automne/Hiver,Reconduit,988.0,Ville,Standard,62475.0,Bottines et boots,...,6.0,3.0,54.90,0.0,Bout pointu,NaN,NaN,NaN,False,Prenez votre pointure habituelle
3,515679.0,Femme,ND,Automne/Hiver,Saisonnier,769.0,Ville,Standard,43983.0,Escarpins,...,0.0,13.0,34.50,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle
4,1025246.0,Femme,ND,Automne/Hiver,Saisonnier,1244.0,ND,Standard,81493.0,Bottines et boots,...,8.0,4.0,43.76,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle


In [149]:
products.SizeAdviceDescription.value_counts()

Prenez votre pointure habituelle                                                                                                                                      262190
Ce modÃ¨le chausse normalement                                                                                                                                        105552
Prenez une taille en dessous de votre pointure habituelle                                                                                                              50620
Prenez une taille au-dessus de votre pointure habituelle                                                                                                               40992
ModÃ¨le Ã©troit, convient aux pieds fins                                                                                                                               20956
ModÃ¨le confortable, convient aux pieds larges                                                                                         

# Comprehension of the Data

In [3]:
print("We will train our algorithm based on a set of %d orders, each with %d features."%(X_train.shape[0],X_train.shape[1]))
print("Then we will test it on a set of %d orders with the same number of features."%(X_test.shape[0]))
print("\nThe customers set consists of %d labels:" %(customers.shape[1]))
print(customers.columns)
print("\nThe products set consists of %d labels:" %(products.shape[1]))
print(products.columns)
print("\nThe x_training set consists of %d labels:" %(X_train.shape[1]))
print(X_train.columns)
print("\nThe y_training set consists of %d labels:" %(y_train.shape[1]))
print(y_train.columns)
print("\nThere are exactly %d orders not returned and %d orders returned" %((y_train['ReturnQuantityBin'] == 0).sum(),(y_train['ReturnQuantityBin'] == 1).sum() ))

We will train our algorithm based on a set of 1067290 orders, each with 20 features.
Then we will test it on a set of 800468 orders with the same number of features.

The customers set consists of 5 labels:
Index(['CustomerId', 'CountryISOCode', 'BirthDate', 'Gender',
       'FirstOrderDate'],
      dtype='object')

The products set consists of 25 labels:
Index(['VariantId', 'GenderLabel', 'MarketTargetLabel', 'SeasonLabel',
       'SeasonalityLabel', 'BrandId', 'UniverseLabel', 'TypeBrand',
       'ProductId', 'ProductType', 'SupplierColor', 'ProductColorId',
       'MinSize', 'MaxSize', 'CalfTurn', 'UpperHeight', 'HeelHeight',
       'PurchasePriceHT', 'IsNewCollection', 'SubtypeLabel',
       'UpperMaterialLabel', 'LiningMaterialLabel', 'OutSoleMaterialLabel',
       'RemovableSole', 'SizeAdviceDescription'],
      dtype='object')

The x_training set consists of 20 labels:
Index(['OrderNumber', 'VariantId', 'LineItem', 'CustomerId',
       'OrderStatusLabel', 'OrderTypelabel', 'Seas

## Modification Customer

In [94]:
birth = pd.DataFrame(np.zeros((customers.shape[0],1)))
firstorder = pd.DataFrame(np.zeros((customers.shape[0],1)))
len = customers.shape[0]
for i in range (len):
    birth.iloc[i]=2018-int(customers['BirthDate'][i][:4])
    firstorder.iloc[i]=2018-int(customers['FirstOrderDate'][i][:4])
    
customers['BirthDate']=birth
customers['FirstOrderDate']=firstorder

In [95]:
birth.describe()
firstorder.describe()

,0
count,780238.000000
mean,4.863953
std,1.694057
min,3.000000
25%,3.000000
50%,5.000000
75%,6.000000
max,13.000000


In [99]:
col = customers['Gender'].eq('Homme').astype('float')
customers['Gender'] = col

In [103]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,14089083.0,SE,39.0,0.0,5.0
1,12862066.0,FR,36.0,0.0,6.0
2,14791699.0,FR,53.0,0.0,5.0
3,10794664.0,FR,52.0,0.0,8.0
4,15268576.0,ES,38.0,0.0,4.0


In [155]:
products.SizeAdviceDescription.value_counts()
print(products['SizeAdviceDescription'][7])

Ce modÃ¨le chausse normalement


## Modification de Products

In [105]:
col = products['SizeAdviceDescription'].eq('Prenez votre pointure habituelle').astype('float')
products['SizeAdviceDescription'] = col

col1 = products['GenderLabel'].eq('Homme').astype('float')
products['GenderLabel'] = col1

del products['MarketTargetLabel']

col2 = products['SeasonalityLabel'].eq('Saisonnier').astype('float')
col3 = products['SeasonalityLabel'].eq('Reconduit').astype('float')
col4 = products['SeasonalityLabel'].eq('Permanent').astype('float')
products['SeasonalityLabel'] = 1.0*col2+2.0*col3+3.0*col4

col5 = products['SeasonLabel'].eq('Automne/Hiver').astype('float')
products['SeasonLabel'] = col5

del products['UniverseLabel']

col6 = products['TypeBrand'].eq('Standard').astype('float')
col7 = products['TypeBrand'].eq('Luxe').astype('float')
col8 = products['TypeBrand'].eq('Super Luxe').astype('float')
products['TypeBrand'] = 1.0*col6+2.0*col7+3.0*col8

col9 = products['ProductType'].eq('Baskets').astype('float')
col10 = products['ProductType'].eq('Bottines et boots').astype('float')
col11 = products['ProductType'].eq('Escarpins').astype('float')
col12 = products['ProductType'].eq('Chaussures de sport').astype('float')
col13 = products['ProductType'].eq('Bottines et boots').astype('float')
col14 = products['ProductType'].eq('Bottes').astype('float')
col15 = products['ProductType'].eq(products['ProductType'][100001]).astype('float') #chaussures à lacets
products['ProductType'] = 1.0*col9+2.0*col10+3.0*col11+4.0*col12+5.0*col13+6.0*col14+7.0*col15

del products['SupplierColor']

col16 = products['RemovableSole'].eq('True').astype('float')
products['RemovableSole'] = col16

products.head()

,VariantId,GenderLabel,SeasonLabel,SeasonalityLabel,BrandId,TypeBrand,ProductId,ProductType,ProductColorId,MinSize,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,728257.0,1.0,1.0,1.0,66.0,1.0,17267.0,1.0,103627.0,41.0,...,NaN,0.0,30.50,0.0,Montantes,NaN,NaN,NaN,0.0,0.0
1,806356.0,0.0,1.0,1.0,842.0,1.0,30824.0,1.0,114192.0,38.0,...,0.0,0.0,43.00,0.0,Montantes,NaN,NaN,NaN,0.0,1.0
2,768790.0,0.0,1.0,2.0,988.0,1.0,62475.0,7.0,109185.0,38.0,...,6.0,3.0,54.90,0.0,Bout pointu,NaN,NaN,NaN,0.0,1.0
3,515679.0,0.0,1.0,1.0,769.0,1.0,43983.0,3.0,73966.0,42.0,...,0.0,13.0,34.50,0.0,Bout rond,NaN,NaN,NaN,0.0,1.0
4,1025246.0,0.0,1.0,1.0,1244.0,1.0,81493.0,7.0,147548.0,37.0,...,8.0,4.0,43.76,0.0,Bout rond,NaN,NaN,NaN,0.0,1.0


In [6]:
#print(products['RemovableSole'])
#codebis = products['UpperMaterialLabel'].eq(products['UpperMaterialLabel'][0]).astype('int')
#print(codebis.value_counts())

In [106]:
products.dtypes

VariantId                float64
GenderLabel              float64
SeasonLabel              float64
SeasonalityLabel         float64
BrandId                  float64
TypeBrand                float64
ProductId                float64
ProductType              float64
ProductColorId           float64
MinSize                  float64
MaxSize                  float64
CalfTurn                 float64
UpperHeight              float64
HeelHeight               float64
PurchasePriceHT          float64
IsNewCollection          float64
SubtypeLabel              object
UpperMaterialLabel        object
LiningMaterialLabel       object
OutSoleMaterialLabel      object
RemovableSole            float64
SizeAdviceDescription    float64
dtype: object

## Preprocessing X_train

In [107]:
index = pd.DataFrame({'index':np.arange(X_train.shape[0])})
X_train['index']=index # Evite les duplicata
X_train.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode,index
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000,0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000,1
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,Plein Tarif,1,1,"13,17",2011-10-26 12:11:38,2011-10-26 17:48:00,1,0.0,77700,2
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,Plein Tarif,2,1,"56,4",2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600,3
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,Plein Tarif,2,1,37,2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600,4


In [108]:
col = X_train['OrderStatusLabel'].eq('Expédié').astype('float')
X_train['OrderStatusLabel'] = col

col1 = X_train['OrderTypelabel'].eq('DIRECT').astype('float')
X_train['OrderTypelabel'] = col1

col2 = X_train['SeasonLabel'].eq('Automne/Hiver').astype('float')
X_train['SeasonLabel'] = col2

col3 = X_train['PayementModeLabel'].eq('Carte bancaire').astype('float')
col4 = X_train['PayementModeLabel'].eq('PayPal').astype('float')
col5 = X_train['PayementModeLabel'].eq('DotPay').astype('float')
col6 = X_train['PayementModeLabel'].eq('Gratuit').astype('float')
col7 = X_train['PayementModeLabel'].eq('Chèque').astype('float')
X_train['PayementModeLabel'] = 1.0*col3 + 2.0*col4 + 3.0*col5 + 4.0*col6 + 5.0*col7

col8 = X_train['CustomerTypeLabel'].eq('Fidélisé').astype('float')
X_train['CustomerTypeLabel'] = col8

del X_train['DeviceTypeLabel']

col9 = X_train['PricingTypeLabel'].eq('Plein Tarif').astype('float')
X_train['PricingTypeLabel'] = col9

In [109]:
X_train.head()
X_train.dtypes

OrderNumber            int64
VariantId              int64
LineItem               int64
CustomerId             int64
OrderStatusLabel     float64
OrderTypelabel       float64
SeasonLabel          float64
PayementModeLabel    float64
CustomerTypeLabel    float64
IsoCode               object
PricingTypeLabel     float64
TotalLineItems         int64
Quantity               int64
UnitPMPEUR            object
OrderCreationDate     object
OrderShipDate         object
OrderNumCustomer       int64
IsOnSale             float64
BillingPostalCode     object
index                  int64
dtype: object

In [110]:
index = pd.DataFrame({'index':np.arange(y_train.shape[0])})
y_train['index']=index # Evite les duplicata, va servir pour le join avec X
y_train.head()

,OrderNumber,LineItem,ReturnQuantity,ReturnQuantityBin,index
0,73521754,1,0.0,0.0,0
1,73521754,2,0.0,0.0,1
2,73525226,1,1.0,1.0,2
3,73529009,1,1.0,1.0,3
4,73529009,2,1.0,1.0,4


In [111]:
X_Product = X_train.merge(products,how='left',on='VariantId')
X_Product = X_Product.sort_values(by='index')
#X_Product = X_Product[:300000] #Reduce computing
#X_Product = preprocessing.scale(X_Product)
X_Product.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel_x,PayementModeLabel,CustomerTypeLabel,IsoCode,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,73521754,439729,1,12443972,1.0,1.0,1.0,1.0,0.0,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73521754,440174,2,12443972,1.0,1.0,1.0,1.0,0.0,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73525226,494501,1,12443958,1.0,1.0,1.0,1.0,0.0,FR,...,12.0,8.0,15.5,0.0,Bout rond,NaN,NaN,NaN,0.0,1.0
3,73529009,439590,1,12443946,1.0,1.0,1.0,1.0,0.0,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73529009,559476,2,12443946,1.0,1.0,1.0,1.0,0.0,FR,...,5.0,9.0,37.0,0.0,Bout rond,Cuir,NaN,NaN,0.0,1.0


In [112]:
X_data = X_Product.merge(customers,how='left',on='CustomerId')
X_data = X_data.sort_values(by='index')
X_data.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel_x,PayementModeLabel,CustomerTypeLabel,IsoCode,...,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,73521754,439729,1,12443972,1.0,1.0,1.0,1.0,0.0,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,60.0,0.0,7.0
1,73521754,440174,2,12443972,1.0,1.0,1.0,1.0,0.0,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,60.0,0.0,7.0
2,73525226,494501,1,12443958,1.0,1.0,1.0,1.0,0.0,FR,...,Bout rond,NaN,NaN,NaN,0.0,1.0,FR,46.0,0.0,7.0
3,73529009,439590,1,12443946,1.0,1.0,1.0,1.0,0.0,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,48.0,0.0,7.0
4,73529009,559476,2,12443946,1.0,1.0,1.0,1.0,0.0,FR,...,Bout rond,Cuir,NaN,NaN,0.0,1.0,FR,48.0,0.0,7.0


In [113]:
X_data.shape

(1067290, 45)

In [114]:
y_Product = pd.merge(y_train,X_data,how='left',on="index")
y_Product = y_Product.sort_values(by='index')
#y_Product = y_Product[:300000]
y_Product.head()

,OrderNumber_x,LineItem_x,ReturnQuantity,ReturnQuantityBin,index,OrderNumber_y,VariantId,LineItem_y,CustomerId,OrderStatusLabel,...,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,73521754,1,0.0,0.0,0,73521754,439729,1,12443972,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,60.0,0.0,7.0
1,73521754,2,0.0,0.0,1,73521754,440174,2,12443972,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,60.0,0.0,7.0
2,73525226,1,1.0,1.0,2,73525226,494501,1,12443958,1.0,...,Bout rond,NaN,NaN,NaN,0.0,1.0,FR,46.0,0.0,7.0
3,73529009,1,1.0,1.0,3,73529009,439590,1,12443946,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,48.0,0.0,7.0
4,73529009,2,1.0,1.0,4,73529009,559476,2,12443946,1.0,...,Bout rond,Cuir,NaN,NaN,0.0,1.0,FR,48.0,0.0,7.0


In [16]:
#C = X_Product.corr()
#print((C))

In [115]:
index = pd.DataFrame({'index':np.arange(X_test.shape[0])})
X_test['index']=index # Evite les duplicata
X_test_Product = X_test.merge(products,how='left',on='VariantId')
X_test_Product = X_test_Product.sort_values(by='index')
X_test_Product.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel_x,PayementModeLabel,CustomerTypeLabel,IsoCode,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,90607166,754279,1,14369241,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Fidélisé,GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90621016,1043714,1,16121040,Expédié,DIRECT,Automne/Hiver,Klarna_Invoice,Fidélisé,DE,...,3.0,2.0,32.700000,0.0,Bout rond,NaN,NaN,NaN,0.0,1.0
2,90630628,302486,1,16271310,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90641377,1300698,1,16271314,Expédié,DIRECT,Automne/Hiver,PayPal,Nouveau,DE,...,7.0,3.0,35.900002,0.0,Bout rond,NaN,NaN,NaN,0.0,0.0
4,90652456,1392714,1,16251703,Expédié,DIRECT,Automne/Hiver,BankTransfer_IBAN,Nouveau,ES,...,NaN,2.0,23.980000,0.0,Montantes,NaN,NaN,NaN,0.0,1.0


In [116]:
X_test_data = X_test_Product.merge(customers,how='left',on='CustomerId')
X_test_data = X_test_data.sort_values(by='index')

In [19]:
X_test_data.shape , X_test.shape

((800468, 46), (800468, 21))

# Defining a feature transformation

In [127]:
features = ["OrderNumber", "VariantId", "LineItem", "CustomerId",
       "OrderStatusLabel", "OrderTypelabel", "SeasonLabel",
       "PayementModeLabel", "CustomerTypeLabel", "IsoCode", "DeviceTypeLabel",
       "PricingTypeLabel", "TotalLineItems", "Quantity",
       "OrderCreationDate", "OrderShipDate", "OrderNumCustomer", "IsOnSale",
       "BillingPostalCode"]

In [21]:
full_features = X_data.columns.values
full_features

array(['OrderNumber', 'VariantId', 'LineItem', 'CustomerId',
       'OrderStatusLabel', 'OrderTypelabel', 'SeasonLabel_x',
       'PayementModeLabel', 'CustomerTypeLabel', 'IsoCode',
       'PricingTypeLabel', 'TotalLineItems', 'Quantity', 'UnitPMPEUR',
       'OrderCreationDate', 'OrderShipDate', 'OrderNumCustomer',
       'IsOnSale', 'BillingPostalCode', 'index', 'GenderLabel',
       'SeasonLabel_y', 'SeasonalityLabel', 'BrandId', 'TypeBrand',
       'ProductId', 'ProductType', 'ProductColorId', 'MinSize', 'MaxSize',
       'CalfTurn', 'UpperHeight', 'HeelHeight', 'PurchasePriceHT',
       'IsNewCollection', 'SubtypeLabel', 'UpperMaterialLabel',
       'LiningMaterialLabel', 'OutSoleMaterialLabel', 'RemovableSole',
       'SizeAdviceDescription', 'CountryISOCode', 'BirthDate', 'Gender',
       'FirstOrderDate'], dtype=object)

In [117]:
F = ["OrderCreationDate","OrderNumber","VariantId", "CustomerId","OrderCreationDate",
     "OrderShipDate","BillingPostalCode","index","ProductColorId","ProductId","BrandId"]

In [118]:
def funk_mask(d,masked_features):
    " Defining a simple mask over the input data "
    columns_ext = masked_features
    X1 = d.loc[:,[xx for xx in d.columns if xx not in columns_ext]]
    print("1")
    g = lambda x: x.replace(",",".")
    print("2")
    X1.UnitPMPEUR = map(np.float64,(map(g,X1.UnitPMPEUR)))
    print("3")
    columns2bin = [x for x in X1.columns if X1[x].dtype == np.dtype('O')]
    print("4")
    X2 = pd.get_dummies(X1.loc[:,columns2bin])
    print("5")
    X1 = X1.loc[:,[xx for xx in X1.columns if xx not in columns2bin]]
    print("6")
    res = pd.concat([X1,X2],axis=1)
    print("7")
    res = res.fillna(0)
    return(res)

# Applying the mask

In [128]:
x1 = funk_mask(X_data,features)
x2 = funk_mask(X_test_data,features)
seleckt_columns = np.intersect1d(x1.columns,x2.columns)
x1 = x1.loc[:,seleckt_columns]
x2 = x2.loc[:,seleckt_columns]

1
2
3
4
5
6
7
1
2
3
4
5
6
7


# Supervised learning : Logistic regression model

In [129]:
clf = RandomForestClassifier()
clf.fit(x1.iloc[:100000], y_Product.ReturnQuantityBin[:100000])
y_tosubmit = clf.predict_proba(x2.loc[:,x1.columns])
score_forest = np.mean(cross_val_score(clf,x1.iloc[:100000], y_Product.ReturnQuantityBin[:100000],cv=10))
print(score_forest)

0.612486651261


In [41]:
print(y_tosubmit[:,1].shape)
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')

(800468,)


# Score of our prediction : on the train

In [125]:
yres = clf.predict_proba(x1.loc[:100000,x1.columns])
print(roc_auc_score(y_Product.ReturnQuantityBin.iloc[:100001],yres[:,1]))

# Submission to the system
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')

0.914423460525
